In [3]:
import tensorflow as tf
INPUT_NODE=784
OUTPUT_NODE=10
IMAGE_SIZE=28
NUM_CHANNELS=1
NUM_LABELS=10

CONV1_DEEP=32
CONV1_SIZE=5
CONV2_DEEP=64
CONV2_SIZE=5
FC_SIZE=512


In [4]:
def inference(input_tensor,train,regularizer):
    with tf.variable_scope('layer1-conv1'):
        conv1_weights=tf.get_variable('weights',[CONV1_SIZE,CONV1_SIZE,NUM_CHANNELS,CONV1_DEEP],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv1_biases=tf.get_variable('bias',[CONV1_DEEP],initializer=tf.constant_initializer(0.0))
        conv1=tf.nn.conv2d(input_tensor,conv1_weights,strides=[1,1,1,1],padding='SAME')
        relu1=tf.nn.relu(tf.nn.bias_add(conv1,conv1_biases))
    with tf.name_scope('layer2-pool1'):
        pool1=tf.nn.max_pool(relu1,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
    with tf.variable_scope('layer3-conv2'):
        conv2_weights=tf.get_variable('weights',[CONV2_SIZE,CONV2_SIZE,CONV1_DEEP,CONV2_DEEP],initializer=tf.truncated_normal_initializer(stddev=0.1))
        conv2_biases=tf.get_variable('bias',[CONV2_DEEP],initializer=tf.constant_initializer(0.0))
        conv2=tf.nn.conv2d(pool1,conv2_weights,strides=[1,1,1,1],padding='SAME')
        relu2=tf.nn.relu(tf.nn.bias_add(conv2,conv2_biases))
    with tf.name_scope('layer4-pool2'):
        pool2=tf.nn.max_pool(relu2,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
        pool_shape=pool2.get_shape().as_list()
        nodes=pool_shape[1]*pool_shape[2]*pool_shape[3]
        reshaped=tf.reshape(pool2,[pool_shape[0],nodes])
        
    with tf.variable_scope('layer5-fc1'):
        fc1_weights=tf.get_variable('weights',[nodes,FC_SIZE],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer!=None):
            tf.add_to_collection('losses',regularizer(fc1_weights))
        fc1_biases=tf.get_variable('bias',[FC_SIZE],initializer=tf.constant_initializer(0.0))
        fc1=tf.nn.relu(tf.matmul(reshaped,fc1_weights)+fc1_biases)
        if(train):
            fc1=tf.nn.dropout(fc1,0.5)
    with tf.variable_scope('layer6-fc2'):
        fc2_weights=tf.get_variable('weights',[FC_SIZE,NUM_LABELS],initializer=tf.truncated_normal_initializer(stddev=0.1))
        if(regularizer!=None):
            tf.add_to_collection('losses',regularizer(fc2_weights))
        fc2_biases=tf.get_variable('bias',[NUM_LABELS],initializer=tf.constant_initializer(0.0))
        logit=tf.matmul(fc1,fc2_weights)+fc2_biases
    return logit


In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import os
import numpy as np

#定义神经网络的相关参数
BATCH_SIZE=100
LEARNING_RATE_BASE=0.01
LEARNING_RATE_DECAY=0.99
REGULARIZATION_RATE=0.0001
TRAINING_STEPS=1000
MOVING_AVERAGE_DECAY=0.99
MODEL_SAVE_PATH='./bb/'
MODEL_NAME='lemodel'


def train(mnist):
    x=tf.placeholder(tf.float32,[BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS],name='X')
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='Y')
    regularizer=tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y=inference(x,True,regularizer)
    global_step=tf.Variable(0,trainable=False)
    
    # 定义损失函数、学习率、滑动平均操作以及训练过程。
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_averages_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(
        LEARNING_RATE_BASE,
        global_step,
        mnist.train.num_examples / BATCH_SIZE, LEARNING_RATE_DECAY,
        staircase=True)

    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_averages_op]):
        train_op = tf.no_op(name='train')
    
    #初始化持久化类
    saver=tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        for i in range(TRAINING_STEPS):
            xs,ys=mnist.train.next_batch(BATCH_SIZE)
            reshaped_xs=np.reshape(xs,(BATCH_SIZE,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS))
            _,loss_value,step=sess.run([train_op,loss,global_step],feed_dict={x:reshaped_xs,y_:ys})
            if(i%100==0):
                print("After %d training step(s) loss on training batch is %g."%(step,loss_value))
                saver.save(sess,os.path.join(MODEL_SAVE_PATH,MODEL_NAME),global_step)
                
def main():
    mnist=input_data.read_data_sets('/tmp/data/',one_hot=True)
    train(mnist)
if __name__=='__main__':
    main()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
After 1 training step(s) loss on training batch is 6.60508.
After 101 training step(s) loss on training batch is 1.86237.
After 201 training step(s) loss on training batch is 1.17289.
After 301 training step(s) loss on training batch is 1.09287.
After 401 training step(s) loss on training batch is 0.936803.
After 501 training step(s) loss on training batch is 1.00035.
After 601 training step(s) loss on training batch is 0.849522.
After 701 training step(s) loss on training batch is 0.810001.
After 801 training step(s) loss on training batch is 0.757437.
After 901 training step(s) loss on training batch is 0.887816.


In [13]:
import time
import math
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x=tf.placeholder(tf.float32,[
            mnist.test.num_examples,
            IMAGE_SIZE,
            IMAGE_SIZE,
            NUM_CHANNELS],name='X_input')
        y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y_input')
        validate_feed={x:mnist.test.images,y_:mnist.test.labels}
        global_step=tf.Variable(0,trainable=False)
        regularizer=tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
        y=inference(x,False,regularizer)
        correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        
        variable_averages=tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver=tf.train.Saver(variables_to_restore)
        n=math.ceil(mnist.test.num_examples/mnist.test.num_examples)
        for i in range(n):
            with tf.Session() as sess:
                ckpt=tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess,ckpt.model_checkpoint_path)
                    global_step=ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    xs,ys=mnist.test.next_batch(mnist.test.num_examples)
                    reshaped_xs=np.reshape(xs,(mnist.test.num_examples,IMAGE_SIZE,IMAGE_SIZE,NUM_CHANNELS))
                    accuracy_score=sess.run(accuracy,feed_dict={x:reshaped_xs,y_:ys})
                    print("After %s training step(s),test accuracy=%g"%(global_step,accuracy_score))
                else:
                    print('No check file found')
                    return
def main():
    mnist=input_data.read_data_sets('/tmp/data/')
    evaluate(mnist)
if __name__=='__main__':
    main()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./bb/lemodel-901


ValueError: Cannot feed value of shape (10000,) for Tensor 'y_input:0', which has shape '(?, 10)'

In [6]:
import time
import math
import tensorflow as tf
import numpy as np

INPUT_NODE=784
OUTPUT_NODE=10
IMAGE_SIZE=28
NUM_CHANNELS=1
NUM_LABELS=10

CONV1_DEEP=32
CONV1_SIZE=5
CONV2_DEEP=64
CONV2_SIZE=5
FC_SIZE=512


#定义神经网络的相关参数
BATCH_SIZE=100
LEARNING_RATE_BASE=0.01
LEARNING_RATE_DECAY=0.99
REGULARIZATION_RATE=0.0001
TRAINING_STEPS=1000
MOVING_AVERAGE_DECAY=0.99
MODEL_SAVE_PATH='./bb/'
MODEL_NAME='lemodel'



from tensorflow.examples.tutorials.mnist import input_data
def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [
            mnist.test.num_examples,
            IMAGE_SIZE,
            IMAGE_SIZE,
            NUM_CHANNELS],
                           name='x-input')
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
        validate_feed = {x: mnist.test.images, y_: mnist.test.labels}
        global_step = tf.Variable(0, trainable=False)

        regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
        y = inference(x, False, regularizer)
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)

        #n = math.ceil(mnist.test.num_examples / LeNet5_train.BATCH_SIZE)
        n = math.ceil(mnist.test.num_examples / mnist.test.num_examples)
        for i in range(n):
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    xs, ys = mnist.test.next_batch(mnist.test.num_examples)
                    #xs, ys = mnist.test.next_batch(LeNet5_train.BATCH_SIZE)
                    reshaped_xs = np.reshape(xs, (
                        mnist.test.num_examples,
                        IMAGE_SIZE,
                        IMAGE_SIZE,
                        NUM_CHANNELS))
                    accuracy_score = sess.run(accuracy, feed_dict={x:reshaped_xs, y_:ys})
                    print("After %s training step(s), test accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return

# 主程序
def main(argv=None):
    mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)
    evaluate(mnist)

if __name__ == '__main__':
    main()

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from ./bb/lemodel-901
After 901 training step(s), test accuracy = 0.9559
